# Restrictions: Did they make a difference?

While we can't say if the measures had a causal impact just by looking at plots, we can see if new restrictions coincide with slowdowns in incidence rate growth.


In [119]:
## load modules

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

# to display offline interactive plots
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# mapping library
import cufflinks as cf

# to export plots to html
import plotly.io as pio

# hide annoying repeated deprec warnings (statsmodel issue)
import warnings
warnings.simplefilter('once', category=UserWarning)

# load my data processing modules
import process_kpi as kpi

# fix slowness issue? See https://github.com/microsoft/vscode-python/issues/10998
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [120]:
## plotting variables

# get the data
kpi_df = kpi.create_kpi_df()

# key dates for restrictions
paris_dates = dict(masks='2020-08-28', alerte_renf='2020-09-26', alerte_max='2020-10-06')
mars_dates = dict(masks='2020-08-26', alerte_max='2020-09-28', restos_reopen='2020-10-05')

# dicts for vertical lines
paris_vl = [dict(x=paris_dates['masks'], dash='dot'),
            dict(x=paris_dates['alerte_renf'], dash='dash'),
           dict(x=paris_dates['alerte_max'])]

mars_vl = [dict(x=mars_dates['masks'], dash='dot'),
           dict(x=mars_dates['alerte_max']),
          dict(x=mars_dates['restos_reopen'], dash='dashdot')]

metrics = ['incid_tous', 'incid_70+', 'rea%']

hover_msg = '(Hover for info, click legend to hide/show items.)'

Saved to data/kpi_2020_10_10.csv
Saved to data/kpi_2020_10_10.pkl


## Paris - in aggregate

In [121]:
# timeline of paris alert indicators

dep = 'Paris'
q = "libelle_dep==@dep"
plot_df = kpi_df.query(q).set_index(['libelle_dep', 'jour'])[metrics].unstack(0).dropna()
plot_df.columns = metrics

title = "<b>Restrictions in {}: at first glance, little impact</b><br>{}".format(dep, hover_msg)
vlines = paris_vl

print("=== Restriction dates for {} ===\n ".format(dep))
for key in paris_dates.keys():
    print(key +": " + paris_dates[key])
    
    fig = plot_df.iplot(vline=vlines,
              title=title, asFigure=True)
    
fig.show()

=== Restriction dates for Paris ===
 
masks: 2020-08-28
alerte_renf: 2020-09-26
alerte_max: 2020-10-06


`incid_tous` slows down slightly after masks are imposed. Moving to Alerte renforcée initially slowed the incidence rate, but it didin't last long. 

If you isolate `rea%`, it looks like declaring Alerte maximale might be helping.

It gets more interesting when look at incidence rate by age group. (There's no age ranges provided for in the publicly-available `rea` counts)

## Paris - by age group

In [122]:
# get incidence rates by dept & age group

dept_age_df = kpi.pt.create_dept_age_df()
dept_age_df = dept_age_df.rename(columns={'pos_100k':'incid_tous'}) # for consistency

In [123]:
# now by plot by age group

title = "{} {}".format(dep, metric)
vlines = paris_vl

title = "<b>Restrictions in {}: cases for 20 & 30-year-olds slowed after imposing masks</b><br>{}".format(dep, hover_msg)
plot_df = dept_age_df.query(q).set_index(['jour', 'age_range'])[metric].unstack().rolling(7).mean()


fig = plot_df.iplot(vline=vlines,
              colorscale='gnbu',
              title=title, asFigure=True)

print("=== Restriction dates for {} ===\n ".format(dep))
for key in paris_dates.keys():
    print(key +": " + paris_dates[key])

fig.show()



=== Restriction dates for Paris ===
 
masks: 2020-08-28
alerte_renf: 2020-09-26
alerte_max: 2020-10-06


In Paris, imposing masks coincides with dramatically slowed cases for the 20-39 age group for about 2 weeks. 

By contrast, Alerte renforcée coincides with only a very short dip in this age group. It did however precede a week-long slowdown for the 50-69 age group before skyrocketing.

## Marseille - in aggregate

In [124]:
# marseille timeline of alert indicators
dep = 'Bouches-du-Rhône'
q = "libelle_dep==@dep"

title = "<b>Restrictions in {}: a jump from reopening restos?</b><br>{}".format(dep, hover_msg)
plot_df = kpi_df.query(q).set_index(['libelle_dep', 'jour'])[metrics].unstack(0).dropna()
plot_df.columns = metrics
vlines = mars_vl

fig = plot_df.iplot(vline=vlines,
              title=title, asFigure=True)

print("=== Restriction dates for {} ===\n ".format(dep))
for key in mars_dates.keys():
    print(key +": " + mars_dates[key])

fig.show()

=== Restriction dates for Bouches-du-Rhône ===
 
masks: 2020-08-26
alerte_max: 2020-09-28
restos_reopen: 2020-10-05


In Marseille, the potential impact on `incid_tous` of imposing masks in late August is even harder to discern than in Paris. 

On the other hand, there seems to correlation with late September closures of bars, cafes & restaurants as a result of the Alerte maximale status & the flattening of both `incid_tous` & `incid_70`...until restos were allowed to reopen, with new safety measures, a week later. 

Of course, correlation does not mean causation!



## Marseille - by age group

In [125]:
# now by age group
title = "<b>Restrictions in {}: masks coincide with slowdown for 20-39 year-olds</b><br>{}".format(dep, hover_msg)
vline = mars_vl

plot_df = dept_age_df.query(q).set_index(['jour', 'age_range'])[metric].unstack().rolling(7).mean()

fig = plot_df.iplot(vline=vline,
       colorscale='gnbu',
       title=title, asFigure=True)

print("=== Restriction dates for {} ===\n ".format(dep))
for key in mars_dates.keys():
    print(key +": " + mars_dates[key])


fig.show()

=== Restriction dates for Bouches-du-Rhône ===
 
masks: 2020-08-26
alerte_max: 2020-09-28
restos_reopen: 2020-10-05


Imposing masks coincides with a dramatic slowddown in incidence rates for 20-somethings, and a flattening for 30-somethings.

On the other hand, re-opening restaurants in early October correlates with a steep uptick in cases for the 70+ age group.